In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*

In [ ]:
// Set verbose level
GlobalConstants.setVerbose(VerboseLevel.STD)

// Create model
val model = Network("model")

// Create nodes
val P1 = Place(model, "P1")
val T1 = Transition(model, "T1")

// Create closed class with 1 job, reference station is P1
val jobclass = ClosedClass(model, "Class1", 1, P1, 0)

In [ ]:
// Add transition modes

// Mode 1: Exponential distribution with mean 1.0
val mode1 = T1.addMode("Mode1")
T1.setDistribution(mode1, Exp.fitMean(1.0))
T1.setEnablingConditions(mode1, jobclass, P1, 1)
T1.setFiringOutcome(mode1, jobclass, P1, 1)

// Mode 2: Erlang distribution with mean 1.0 and order 2
val mode2 = T1.addMode("Mode2")
T1.setDistribution(mode2, Erlang.fitMeanAndOrder(1.0, 2))
T1.setEnablingConditions(mode2, jobclass, P1, 1)
T1.setFiringOutcome(mode2, jobclass, P1, 1)

// Mode 3: HyperExponential distribution with mean 1.0 and SCV = 4.0
val mode3 = T1.addMode("Mode3")
T1.setDistribution(mode3, HyperExp.fitMeanAndSCV(1.0, 4.0))
T1.setEnablingConditions(mode3, jobclass, P1, 1)
T1.setFiringOutcome(mode3, jobclass, P1, 1)

In [ ]:
// Set up routing
val R = model.initRoutingMatrix()
R.set(jobclass, jobclass, P1, T1, 1.0)
R.set(jobclass, jobclass, T1, P1, 1.0)
model.link(R)

In [ ]:
// Set initial state - 1 token in P1
P1.setState(Matrix(doubleArrayOf(jobclass.population.toDouble())))

In [ ]:
// Solve with JMT
val options = SolverOptions(SolverType.JMT)
options.seed = 23000

val solver = JMT(model, options)
val avgTable = solver.avgTable
avgTable.print()